### <font color='#4285f4'>Overview</font>

This demo analyzes our text-to-video and creates "YouTube" Shorts from the longer content. We provide Gemini with ABCD best practices, relevant research, and the video itself. Gemini then identifies the most exciting segment suitable for a Short, generating a new voice over script for that portion. Finally, text-to-speech technology produces the audio, completing the Short.

Process Flow:

1. Download the full marketing video without audio
2. Download ABCD best practices PDF
3. Download a research paper about "shorts"
4. Prompt Gemini to watch the video, read the best practices, and ask for the timestamps of a YouTube short along with a voice over.
5. Extract the video based upon Gemini's outputted timestamps
6. Use text-to-speech to generate the voice over.
7. Merge the video and the voice over.

Notes:

1. The notebook does merge the videos, but the quality is much better when you merge with a high quality video tool. The text-to-speech does not always align perfectly with the video.
2. You can also adjust the speed of the text-to-speech to control how fast the words are spoken.

Cost:
* Low: Gemini, BigQuery
* Medium: Remember to stop your Colab Enterprise Notebook Runtime

Author: 
* Adam Paternostro

In [ ]:
# Architecture Diagram
from IPython.display import Image
Image(url='https://storage.googleapis.com/data-analytics-golden-demo/chocolate-ai/v1/Artifacts/Campaign-Assets-Video-Create-Shorts-Architecture.png', width=1200)

### <font color='#4285f4'>Video Walkthrough</font>

[![Video](../images/adam-paternostro-video.png)](https://storage.googleapis.com/data-analytics-golden-demo/chocolate-ai/v1/Videos/Campaign-Assets-Video-Create-Shorts.mp4)


In [ ]:
from IPython.display import HTML

HTML("""
<video width="800" height="600" controls>
  <source src="https://storage.googleapis.com/data-analytics-golden-demo/chocolate-ai/v1/Videos/Campaign-Assets-Video-Create-Shorts.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

### <font color='#4285f4'>License</font>

```
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

Author: Adam Paternostro

### <font color='#4285f4'>Initialize</font>

In [ ]:
from PIL import Image
from IPython.display import HTML
from IPython.display import Audio
from functools import reduce
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2
import random
import time
import datetime
import base64
import random
import datetime

import logging
from tenacity import retry, wait_exponential, stop_after_attempt, before_sleep_log, retry_if_exception

In [ ]:
# Set these (run this cell to verify the output)

bigquery_location = "${bigquery_location}"
region = "${region}"
location = "${location}"
storage_account = "${chocoate_ai_bucket}"
dataset_name = "${bigquery_chocoate_ai_dataset}"
public_storage_storage_account = "data-analytics-golden-demo"

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as desired
formatted_date = now.strftime("%Y-%m-%d-%H-%M")

# Get some values using gcloud
project_id = !(gcloud config get-value project)
user = !(gcloud auth list --filter=status:ACTIVE --format="value(account)")

if len(project_id) != 1:
  raise RuntimeError(f"project_id is not set: {project_id}")
project_id = project_id[0]

if len(user) != 1:
  raise RuntimeError(f"user is not set: {user}")
user = user[0]

print(f"project_id = {project_id}")
print(f"user = {user}")

### <font color='#4285f4'>Pip installs</font>

In [ ]:
# PIP Installs
import sys

# https://pypi.org/project/moviepy/
!{sys.executable} -m pip install moviepy

### <font color='#4285f4'>Helper Methods</font>

#### restAPIHelper
Calls the Google Cloud REST API using the current users credentials.

In [ ]:
def restAPIHelper(url: str, http_verb: str, request_body: str) -> str:
  """Calls the Google Cloud REST API passing in the current users credentials"""

  import requests
  import google.auth
  import json

  # Get an access token based upon the current user
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
    "Content-Type" : "application/json",
    "Authorization" : "Bearer " + access_token
  }

  if http_verb == "GET":
    response = requests.get(url, headers=headers)
  elif http_verb == "POST":
    response = requests.post(url, json=request_body, headers=headers)
  elif http_verb == "PUT":
    response = requests.put(url, json=request_body, headers=headers)
  elif http_verb == "PATCH":
    response = requests.patch(url, json=request_body, headers=headers)
  elif http_verb == "DELETE":
    response = requests.delete(url, headers=headers)
  else:
    raise RuntimeError(f"Unknown HTTP verb: {http_verb}")

  if response.status_code == 200:
    return json.loads(response.content)
    #image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
  else:
    error = f"Error restAPIHelper -> ' Status: '{response.status_code}' Text: '{response.text}'"
    raise RuntimeError(error)

#### RetryCondition (for retrying LLM calls)

In [ ]:
def RetryCondition(error):
  error_string = str(error)
  print(error_string)

  retry_errors = [
      "RESOURCE_EXHAUSTED",
      "No content in candidate",
      # Add more error messages here as needed
  ]

  for retry_error in retry_errors:
    if retry_error in error_string:
      print("Retrying...")
      return True

  return False

#### Gemini LLM (Pro 1.0 , Pro 1.5)

In [ ]:
@retry(wait=wait_exponential(multiplier=1, min=1, max=60), stop=stop_after_attempt(10), retry=retry_if_exception(RetryCondition), before_sleep=before_sleep_log(logging.getLogger(), logging.INFO))
def GeminiLLM(prompt, model = "gemini-1.5-pro-001", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-1.5-pro-001"
  # model = "gemini-pro" # This does support topK (but the function is more generic)
  # model = "gemini-1.0-pro" # This does not support response_schema

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-pro" or model == "gemini-1.0-pro" or model == "gemini-1.0-pro-vision-001":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": {
          "text": prompt
      },
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Gemini LLM - Multimodal

In [ ]:
@retry(wait=wait_exponential(multiplier=1, min=1, max=60), stop=stop_after_attempt(10), retry=retry_if_exception(RetryCondition), before_sleep=before_sleep_log(logging.getLogger(), logging.INFO))
def GeminiLLM_Multimodal(multimodal_prompt_list, model = "gemini-1.5-pro-001", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-1.5-pro-001"
  # model = "gemini-pro" # This does support topK (but the function is more generic)
  # model = "gemini-1.0-pro" # This does not support response_schema

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-pro" or model == "gemini-1.0-pro" or model == "gemini-1.0-pro-vision-001":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": multimodal_prompt_list
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Download GCS file

In [ ]:
def download_from_gcs(destination_file_name, filename, gcs_storage_bucket, gcs_storage_path):
  # prompt: Write python code to download a blob from a gcs bucket.  do not use the requests method

  from google.cloud import storage

  # The ID of your GCS object
  object_name = gcs_storage_path + filename

  storage_client = storage.Client()

  bucket = storage_client.bucket(gcs_storage_bucket)

  # Construct a client side representation of a blob.
  # Note `Bucket.blob` differs from `Bucket.get_blob` as it doesn't retrieve
  # any content from Google Cloud Storage. As we don't need additional data,
  # using `Bucket.blob` is preferred here.
  blob = bucket.blob(object_name)
  blob.download_to_filename(destination_file_name)

  print(
      "Downloaded storage object {} from bucket {} to local file {}.".format(
          object_name, gcs_storage_bucket, destination_file_name
      )
  )

#### Copy file to GCS

In [ ]:
# This was generated by GenAI

def copy_file_to_gcs(local_file_path, bucket_name, destination_blob_name):
  """Copies a file from a local drive to a GCS bucket.

  Args:
      local_file_path: The full path to the local file.
      bucket_name: The name of the GCS bucket to upload to.
      destination_blob_name: The desired name of the uploaded file in the bucket.

  Returns:
      None
  """

  import os
  from google.cloud import storage

  # Ensure the file exists locally
  if not os.path.exists(local_file_path):
      raise FileNotFoundError(f"Local file '{local_file_path}' not found.")

  # Create a storage client
  storage_client = storage.Client()

  # Get a reference to the bucket
  bucket = storage_client.bucket(bucket_name)

  # Create a blob object with the desired destination path
  blob = bucket.blob(destination_blob_name)

  # Upload the file from the local filesystem
  content_type = ""
  if local_file_path.endswith(".html"):
    content_type = "text/html; charset=utf-8"

  if local_file_path.endswith(".json"):
    content_type = "application/json; charset=utf-8"

  if content_type == "":
    blob.upload_from_filename(local_file_path)
  else:
    blob.upload_from_filename(local_file_path, content_type = content_type)

  print(f"File '{local_file_path}' uploaded to GCS bucket '{bucket_name}' as '{destination_blob_name}.  Content-Type: {content_type}'.")

#### Text-to-Speech

In [ ]:
def TextToSpeechLanguageList(language_code):
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token,
      "x-goog-user-project" : project
  }

  # https://cloud.google.com/text-to-speech/docs/reference/rest/v1/voices/list
  url = f"https://texttospeech.googleapis.com/v1/voices?languageCode={language_code}"

  response = requests.get(url, headers=headers)

  if response.status_code == 200:
    return response.text
  else:
    error = f"Error with language_code:'{language_code}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)

In [ ]:
def TextToSpeech(local_filename, text, language_code, language_code_name, ssml_gender, speaking_rate = 1):
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token,
      "x-goog-user-project" : project
  }

  # https://cloud.google.com/text-to-speech/docs/reference/rest/v1/text/synthesize
  url = f"https://texttospeech.googleapis.com/v1/text:synthesize"

  payload = {
   "input": {
      "text": text
   },
   "voice": {
      "languageCode": language_code,
      "name": language_code_name,
      "ssmlGender": ssml_gender # FEMALE | MALE
   },
   "audioConfig": {
      "audioEncoding": "MP3",
      "speakingRate": speaking_rate,
   }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    audio_data = json.loads(response.content)["audioContent"]
    audio_data = base64.b64decode(audio_data)
    with open(local_filename, "wb") as f:
      f.write(audio_data)
    print(f"Audio generated OK.")
    return local_filename
  else:
    error = f"Error with text:'{text}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)

#### MergeVideoAndAudio and ExtractClip

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip

def MergeVideoAndAudio(video_filename, audio_filename, output_filename):
  # Load the video and audio files
  video = VideoFileClip(video_filename)
  audio = AudioFileClip(audio_filename)

  # Combine the video and audio
  final_clip = video.set_audio(audio)

  # Save the combined video
  final_clip.write_videofile(output_filename)

In [ ]:
from moviepy.editor import *

def ExtractClip(input_video_path, start_time, end_time):
  # Load the video clip
  clip = VideoFileClip(input_video_path)

  # Extract the desired segment
  extracted_clip = clip.subclip(start_time, end_time)

  # Save the extracted clip as a new video file
  extracted_clip.write_videofile(youtube_extracted_clip_video_filename)

#### Helper Functions

In [ ]:
# prompt: python to delete a file even if it does not exist

def delete_file(filename):
  try:
    os.remove(filename)
    print(f"File '{filename}' deleted successfully.")
  except FileNotFoundError:
    print(f"File '{filename}' not found.")

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  #print(json_formatted_str)
  return json_formatted_str

In [ ]:
# prompt: python code to download a pdf from the internet

import requests

def download_http_file(url, filename):
  """Downloads a PDF file from a given URL.

  Args:
      url: The URL of the PDF file to download.
      filename: The name to save the downloaded PDF file as.
  """
  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes

    with open(filename, 'wb') as f:
      f.write(response.content)

    print(f"PDF downloaded successfully to {filename}")

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while downloading the PDF: {e}")

# Example usage:


### <font color='#4285f4'>Download Existing Ad and Best Pratices for Creating YouTube Shorts</font>

In [ ]:
# Download our sample Chocolate A.I. Ads and upload to our project storage account (this way they will be local and we can access via Gemini)

download_from_gcs("story-01-full-video-no-audio.mp4", "full-video-no-audio.mp4", public_storage_storage_account, "chocolate-ai/v1/Campaign-Assets-Text-to-Video-01/story-01/")
download_from_gcs("story-02-full-video-no-audio.mp4", "full-video-no-audio.mp4", public_storage_storage_account, "chocolate-ai/v1/Campaign-Assets-Text-to-Video-01/story-02/")
download_from_gcs("story-03-full-video-no-audio.mp4", "full-video-no-audio.mp4", public_storage_storage_account, "chocolate-ai/v1/Campaign-Assets-Text-to-Video-01/story-03/")

copy_file_to_gcs("story-01-full-video-no-audio.mp4", storage_account, f"chocolate-ai/Campaign-Assets-Video-Create-Shorts/youtube-short-{formatted_date}/story-01-full-video-no-audio.mp4")
copy_file_to_gcs("story-02-full-video-no-audio.mp4", storage_account, f"chocolate-ai/Campaign-Assets-Video-Create-Shorts/youtube-short-{formatted_date}/story-02-full-video-no-audio.mp4")
copy_file_to_gcs("story-03-full-video-no-audio.mp4", storage_account, f"chocolate-ai/Campaign-Assets-Video-Create-Shorts/youtube-short-{formatted_date}/story-03-full-video-no-audio.mp4")


In [ ]:
# https://blog.google/products/ads-commerce/youtube-shorts-ads-select-lineups-abcds/

abcd_filename = "Marketing-Shorts-ABCD-One-Sheeter.pdf"
download_from_gcs(abcd_filename, abcd_filename, public_storage_storage_account, "chocolate-ai/v1/Artifacts/")
copy_file_to_gcs(abcd_filename, storage_account, f"chocolate-ai/Campaign-Assets-Video-Create-Shorts/youtube-short-{formatted_date}/{abcd_filename}")

In [ ]:
# https://arxiv.org/html/2402.18208v1
# Shorts on the Rise: Assessing the Effects of YouTube: Shorts on Long-Form Video Content
arxiv_url = "https://arxiv.org/pdf/2402.18208v1"
arxiv_filename = "Shorts-on-the-Rise-Assessing-the-Effects-of-YouTube.pdf"
download_http_file(arxiv_url, arxiv_filename)
copy_file_to_gcs(arxiv_filename, storage_account, f"chocolate-ai/Campaign-Assets-Video-Create-Shorts/youtube-short-{formatted_date}/{arxiv_filename}")

In [ ]:
print(f"View the GCS bucket: https://console.cloud.google.com/storage/browser/{storage_account}/chocolate-ai/Campaign-Assets-Video-Create-Shorts/youtube-short-{formatted_date}")

### <font color='#4285f4'>YouTube Short - Gemini Prompt</font>

In [ ]:
# Write me the json in  OpenAPI 3.0 schema object for the below object.
# Make all fields required.
#  {
#    "youtube_short_description" : "text",
#    "youtube_short_begin_timestamp" : "text",
#    "youtube_short_end_timestamp" : "text",
#    "youtube_short_explanation" : "text",
#    "youtube_short_voice_over" : "text",
#    "abcd_best_practices_in_short" : ["text"]
#  }
response_schema = {
  "type": "object",
  "required": [
    "youtube_short_description",
    "youtube_short_begin_timestamp",
    "youtube_short_end_timestamp",
    "youtube_short_explanation",
    "youtube_short_voice_over",
    "abcd_best_practices_in_short"
  ],
  "properties": {
    "youtube_short_description": {
      "type": "string"
    },
    "youtube_short_begin_timestamp": {
      "type": "string"
    },
    "youtube_short_end_timestamp": {
      "type": "string"
    },
    "youtube_short_explanation": {
      "type": "string"
    },
    "youtube_short_voice_over": {
      "type": "string"
    },
    "abcd_best_practices_in_short": {
      "type": "array",
      "items": {
        "type": "string"
      }
    }
  }
}

prompt = """I need you to watch the video and find the most interesting exciting part for a YouTube short.
I attached the file {abcd_filename} which contains the best practices for creating a YouTube short.
Incorporate the best practices into the video.
This is for a company called "Chocolate A.I." based in Paris, France.
Select the best 10 seconds of based upon the {abcd_filename}.
Write a 10 second voice over for the segment.
- The voice over shoud be a short, engaging, and memorable.
- The voice over should have an introduction and conclusion.
- The voice over should be 140 to 200 characters.
- The voice over should be 25 to 30 words.

- IMPORTANT:
  - If you end a sentence with "Chocolate A.I." in the "youtube_short_voice_over" field, make sure you place a second period like "Chocolate A.I..".
  - A second period is needed for the voice-to-text to work properly.
  - Example: "In the heart of Paris, a masterpiece takes shape. Chocolate A.I.  Where artistry meets indulgence." should become "In the heart of Paris, a masterpiece takes shape. Chocolate A.I..  Where artistry meets indulgence."

Places the ABCD best practices that will be contained in the short in the "abcd_best_practices_in_short" field.
"""

# You can you any of these 3 files.
marketing_story = "01" # You can use 01, 02 or 03. Try out all three.

marketing_video_name_to_review = f"story-{marketing_story}-full-video-no-audio.mp4"

multimodal_prompt_list = [
    { "text": prompt },
    { "fileData": {  "mimeType": "video/mp4", "fileUri": f"gs://{storage_account}/chocolate-ai/Campaign-Assets-Video-Create-Shorts/youtube-short-{formatted_date}/{marketing_video_name_to_review}" } },
    { "fileData": {  "mimeType": "application/pdf", "fileUri": f"gs://{storage_account}/chocolate-ai/Campaign-Assets-Video-Create-Shorts/youtube-short-{formatted_date}/{abcd_filename}" } },
    { "fileData": {  "mimeType": "application/pdf", "fileUri": f"gs://{storage_account}/chocolate-ai/Campaign-Assets-Video-Create-Shorts/youtube-short-{formatted_date}/Shorts-on-the-Rise-Assessing-the-Effects-of-YouTube.pdf" } }
  ]

youtube_short_response = GeminiLLM_Multimodal(multimodal_prompt_list, response_schema=response_schema)

youtube_short_dict = json.loads(youtube_short_response)

print(PrettyPrintJson(youtube_short_response))

### <font color='#4285f4'>YouTube Short - Extract the Video Short</font>

In [ ]:
youtube_extracted_clip_video_filename = f"youtube-short-extract-clip-story-{marketing_story}.mp4"
youtube_extracted_clip_audio_filename = f"youtube-short-extract-clip-story-{marketing_story}.mp3"
youtube_short_filename                = f"youtube-short-extract-clip-story-{marketing_story}-final.mp4"

ExtractClip(marketing_video_name_to_review, youtube_short_dict['youtube_short_begin_timestamp'], youtube_short_dict['youtube_short_end_timestamp'])

In [ ]:
# prompt: python to play a mp4 in a jupyter notebook
video_mp4 = open(youtube_extracted_clip_video_filename, 'rb').read()
video_url = "data:video/mp4;base64," + base64.b64encode(video_mp4).decode()

In [ ]:
# 16:9 aspect ratio
HTML(f"""
<p>YouTube Short (no audio)</p>
<video width=600 height=337 controls>
      <source src="{video_url}" type="video/mp4">
</video>
""")

### <font color='#4285f4'>YouTube Short - Generate the Audio (Voice Over)</font>

In [ ]:
# Print out a list of language code, select one you want
response_text = TextToSpeechLanguageList("en-gb")
response_json = json.loads(response_text)
print (response_json)
# print(PrettyPrintJson(response_text))

language_code = "en-gb"
language_code_name = "en-GB-Standard-B"
ssml_gender = "MALE"

In [ ]:
# Generate the text-to-speech for each segment
# You can change the speed of the voice [last parameter in TextToSpeech] to either shorted or lengthen the audio lenght.
# Ideally we would generate the voiceover with an exact number of works for 10 seconds

# Output of video files
voiceover_prompt = youtube_short_dict['youtube_short_voice_over']
print(f"Generating: {voiceover_prompt}")

# Text-to-Speech
TextToSpeech(youtube_extracted_clip_audio_filename, voiceover_prompt, language_code, language_code_name, ssml_gender, .95)
display(Audio(youtube_extracted_clip_audio_filename, autoplay=True,rate=16000))
print()

### <font color='#4285f4'>YouTube Short - Merge Video and Audio for Final Video</font>

In [ ]:
# This does a decent job at merging the video and audio, there are alot more settings you can configure, this is just a basic merge

MergeVideoAndAudio(youtube_extracted_clip_video_filename,youtube_extracted_clip_audio_filename,youtube_short_filename)

In [ ]:
# prompt: python to play a mp4 in a jupyter notebook
video_mp4 = open(youtube_short_filename, 'rb').read()
video_url = "data:video/mp4;base64," + base64.b64encode(video_mp4).decode()

In [ ]:
# 16:9 aspect ratio
HTML(f"""
<p>YouTube Short (with Audio)</p>
<video width=600 height=337 controls>
      <source src="{video_url}" type="video/mp4">
</video>
""")

### <font color='#4285f4'>YouTube Short - Upload Artifacts to GCS</font>

In [ ]:
copy_file_to_gcs(youtube_extracted_clip_video_filename,    storage_account, f"chocolate-ai/Campaign-Assets-Video-Create-Shorts/youtube-short-{formatted_date}/{youtube_extracted_clip_video_filename}")
copy_file_to_gcs(youtube_extracted_clip_audio_filename,    storage_account, f"chocolate-ai/Campaign-Assets-Video-Create-Shorts/youtube-short-{formatted_date}/{youtube_extracted_clip_audio_filename}")
copy_file_to_gcs(youtube_short_filename,                   storage_account, f"chocolate-ai/Campaign-Assets-Video-Create-Shorts/youtube-short-{formatted_date}/{youtube_short_filename}")

# Save the prompt so we know how we got this data
with open("gemini_youtube_short_prompt.txt", "w") as f:
  f.write(prompt)
copy_file_to_gcs("gemini_youtube_short_prompt.txt", storage_account, f"chocolate-ai/Campaign-Assets-Video-Create-Shorts/youtube-short-{formatted_date}/gemini_youtube_short_prompt-{marketing_story}.txt")
delete_file("gemini_youtube_short_prompt.txt")

# Save the output of the prompt
with open("gemini_youtube_short_prompt_results.txt", "w") as f:
  f.write(PrettyPrintJson(json.dumps(youtube_short_dict)))
copy_file_to_gcs("gemini_youtube_short_prompt_results.txt", storage_account, f"chocolate-ai/Campaign-Assets-Video-Create-Shorts/youtube-short-{formatted_date}/gemini_youtube_short_prompt_results-{marketing_story}.txt")
delete_file("gemini_youtube_short_prompt_results.txt")

# To view the bucket
print()
print("Click here to view the bucket")
print(f"https://console.cloud.google.com/storage/browser/{storage_account}/chocolate-ai/Campaign-Assets-Video-Create-Shorts/youtube-short-{formatted_date}")

### <font color='#4285f4'>Clean Up</font>

In [ ]:
# Placeholder

### <font color='#4285f4'>Reference Links</font>


- [Google.com](https://www.google.com)